In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import torch


import time
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms


In [ ]:
file_dir = '/content/drive/My Drive/APS360/' #csv file location

with open(file_dir + 'Data_Fixed_2.csv','r') as csvfile:
    data_reader = csv.reader(csvfile)
    data_orig = []
    for row in data_reader:
        data_orig.append(row)

data_matrix = np.array(data_orig)
print(data_matrix[0])

['Rating' 'Working Capital' 'Column2' 'Column3' 'Column4' 'Column5'
 'Quick Ratio' 'Column6' 'Column7' 'Column8' 'Column9' 'EPS' 'Column10'
 'Column11' 'Column12' 'Column13' 'PE Ratio' 'Column14' 'Column15'
 'Column16' 'Column17' 'Short to Long Term Debt' 'Column18' 'Column19'
 'Column20' 'Column21' 'ROE' 'Column22' 'Column23' 'Column24' 'Column25'
 'Current' 'Column26' 'Column27' 'Column28' 'Column29'
 'Net Project Margin' 'Column30' 'Column31' 'Column32' 'Column33'
 'Ave 30 day Volume' 'Ave 6 month Volume' 'Ave 3 month Volume'
 'Volume 5day ave' 'PX 1 Yr Ago' 'PX 6 Months Ago' 'PX 3m ago'
 'PX 1 Day ago']


In [ ]:
"""
train_dl=[]
for i in range(len(train_data)):
   train_dl.append([train_data[i], train_labels[i]])
train_dl= np.array(train_dl)
train_loader = torch.utils.data.DataLoader(train_dl, shuffle=True, batch_size=100)


val_dl=[]
for i in range(len(val_data)):
   val_dl.append([val_data[i], val_labels[i]])
val_dl=np.array(val_dl)
val_loader = torch.utils.data.DataLoader(val_dl, shuffle=True, batch_size=100)

test_dl=[]
for i in range(len(test_data)):
   test_dl.append([test_data[i], test_labels[i]])
test_dl=np.array(test_dl)
test_loader = torch.utils.data.DataLoader(test_dl, shuffle=True, batch_size=100)

print(train_dl)
"""

'\ntrain_dl=[]\nfor i in range(len(train_data)):\n   train_dl.append([train_data[i], train_labels[i]])\ntrain_dl= np.array(train_dl)\ntrain_loader = torch.utils.data.DataLoader(train_dl, shuffle=True, batch_size=100)\n\n\nval_dl=[]\nfor i in range(len(val_data)):\n   val_dl.append([val_data[i], val_labels[i]])\nval_dl=np.array(val_dl)\nval_loader = torch.utils.data.DataLoader(val_dl, shuffle=True, batch_size=100)\n\ntest_dl=[]\nfor i in range(len(test_data)):\n   test_dl.append([test_data[i], test_labels[i]])\ntest_dl=np.array(test_dl)\ntest_loader = torch.utils.data.DataLoader(test_dl, shuffle=True, batch_size=100)\n\nprint(train_dl)\n'

In [ ]:
"""
label_array = data_matrix[1:]
label_array = label_array.astype(np.float)
data_tensor = torch.tensor(label_array)

print(data_tensor[0])
"""

'\nlabel_array = data_matrix[1:]\nlabel_array = label_array.astype(np.float)\ndata_tensor = torch.tensor(label_array)\n\nprint(data_tensor[0])\n'

In [ ]:
"""
train_set,val_set,test_set=torch.utils.data.random_split(data_tensor, [328,41,41])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=41, num_workers=1, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=41, num_workers=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(val_set, batch_size=41, num_workers=1, shuffle=True)

"""

'\ntrain_set,val_set,test_set=torch.utils.data.random_split(data_tensor, [328,41,41])\n\ntrain_loader = torch.utils.data.DataLoader(train_set, batch_size=41, num_workers=1, shuffle=True)\nval_loader = torch.utils.data.DataLoader(val_set, batch_size=41, num_workers=1, shuffle=True)\ntest_loader = torch.utils.data.DataLoader(val_set, batch_size=41, num_workers=1, shuffle=True)\n\n'

In [ ]:
class Stock(nn.Module):
    def __init__(self):
        super(Stock, self).__init__()
        self.layer1 = nn.Linear(48, 15)
        self.layer2 = nn.Linear(15, 8)
        self.layer3 = nn.Linear(8, 5)
    def forward(self, img):
        flattened = img.view(-1, 49)
        activation1 = self.layer1(flattened)
        activation1 = F.relu(activation1)
        activation2 = self.layer2(activation1)
        activation2 = F.relu(activation2)
        activation3 = self.layer3(activation2)
        return activation3


In [ ]:
train_data = data_matrix[1:329,1:]
test_data = data_matrix[329:370,1:]
val_data = data_matrix[370:,1:]

train_labels = data_matrix[1:329,0]
test_labels = data_matrix[329:370,0]
val_labels = data_matrix[370:,0]

train_label_tensor = torch.utils.data.DataLoader(train_labels, batch_size=41, num_workers=1, shuffle=True)
val_label_tensor= torch.utils.data.DataLoader(val_labels, batch_size=41, num_workers=1, shuffle=True)
test_label_tensor= torch.utils.data.DataLoader(test_labels, batch_size=41, num_workers=1, shuffle=True)

def get_accuracy(model, train=False):
    if train:
        data = train_data
        labels = train_label_tensor
    else:
        data = val_data
        labels = val_label_tensor

    correct = 0
    total = 0
    for stock in torch.utils.data.DataLoader(data, batch_size=41):
        output = model(stock)
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += stock.shape[0]
    return correct / total



<class 'torch.utils.data.dataloader.DataLoader'>


In [ ]:
def train(model, data, batch_size=41, num_epochs=1):
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    if data.shape[0] == 328:
        label = train_labels
    else:
        label = val_labels
    iters, losses, train_acc, val_acc = [], [], [], []
 
    # training
    n = 0 # the number of iterations
    for epoch in range(num_epochs):
        for stock in iter(train_loader):
            out = model(stock)             # forward pass
            loss = criterion(out, label) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch
 
            # save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)             # compute *average* loss
            train_acc.append(get_accuracy(model, train=True)) # compute training accuracy 
            val_acc.append(get_accuracy(model, train=False))  # compute validation accuracy
            n += 1
 
    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()
 
    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Train")
    plt.plot(iters, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()
 
    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
    
    
print('done')


done


In [ ]:
stockmodel = Stock()

train(stockmodel,train_data,batch_size=64, num_epochs=10 )

TypeError: ignored